In [ ]:
"""Script to download SkySat assets. Use with caution, make sure 
you stay within the download rate range available. 
Script adapted from https://github.com/planetlabs/planet-client-python/blob/main/examples/data_download_multiple_assets.py
"""
import asyncio
import nest_asyncio
nest_asyncio.apply()
import shutil
from planet import exceptions, Session
import httpx
import os

In [ ]:
# Example Skysat assets
amhara_ids = ['20221222_065703_ss02_u0001', '20221004_111044_ssc6_u0002']

In [ ]:
asset_ids = amhara_ids
item_type = 'SkySatCollect'
asset_type = 'ortho_visual'


async def download_and_validate(client, item_id, item_type_id, asset_type_id):
    """Activate, download, and validate an asset as a single task."""
    # Get asset description
    asset = await client.get_asset(item_type_id, item_id, asset_type_id)

    # Activate asset
    await client.activate_asset(asset)

    # Wait for asset to become active
    asset = await client.wait_asset(asset, callback=print)
    # Download asset
    path = await client.download_asset(asset)

    # Move images to new directory
    new_path = 'new_path' / path
    shutil.move(path, new_path)

async def main():
    """Download and validate assets in parallel."""
    concurrency = 10 # Set the number of concurrent tasks
    max_retries = 3 # Set the maximum number of retries for failed tasks
    retry_exceptions = [
        httpx.ConnectError,
        httpx.ReadError,
        httpx.ReadTimeout,
        httpx.RemoteProtocolError,
        exceptions.BadGateway,
        exceptions.TooManyRequests
    ]
    
    async with Session() as sess:
        client = sess.client('data')
        sem = asyncio.Semaphore(concurrency) # Create a semaphore object
        tasks = [] # Create an empty list to store the tasks
        
        for img in asset_ids:
            # Acquire the semaphore to limit the number of concurrent tasks
            async with sem:
                # Create a new task and add it to the list of tasks
                task = asyncio.create_task(download_and_validate(client,
                                                                  img,
                                                                  item_type,
                                                                  asset_type))
                tasks.append(task)
                
        # Wait for all tasks to complete
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Retry failed tasks
        retry_tasks = []
        for i, result in enumerate(results):
            if isinstance(result, Exception) and isinstance(result, tuple(retry_exceptions)):
                retries = 0
                while retries < max_retries:
                    print(f"Retrying task {i} ({retries + 1}/{max_retries})...")
                    async with sem:
                        task = asyncio.create_task(download_and_validate(client,
                                                                          asset_ids[i],
                                                                          item_type,
                                                                          asset_type))
                        try:
                            result = await task
                            break
                        except Exception as e:
                            if isinstance(e, tuple(retry_exceptions)):
                                retries += 1
                            else:
                                raise
                if isinstance(result, Exception):
                    print(f"Task {i} failed after {max_retries} retries: {result}")
                else:
                    print(f"Task {i} succeeded after {retries + 1} retries.")
        print("All tasks completed.")

if __name__ == '__main__':
    asyncio.run(main())